In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
mohamedkhaled28_full_data_final_path = kagglehub.dataset_download('mohamedkhaled28/full-data-final')
mohamedkhaled28_test_data_test_path = kagglehub.dataset_download('mohamedkhaled28/test-data-test')

print('Data source import complete.')


In [ ]:
import os
import time
import random
import numpy as np
import pandas as pd
import cv2

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier

import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras import backend, layers, metrics
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import (
    Input, Conv2D, UpSampling2D, Concatenate, Dense, GlobalAveragePooling2D,
    MaxPooling2D, BatchNormalization, Dropout, Flatten, LeakyReLU
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import (
    ResNet50, VGG16, NASNetMobile, MobileNetV2, InceptionV3,
    DenseNet121, Xception, EfficientNetB0
)
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import (
    ImageDataGenerator, load_img, img_to_array
)
from tensorflow.keras.utils import (
    Sequence, to_categorical, plot_model
)
from tensorflow.keras.callbacks import (
    EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
)
from tensorflow.keras.initializers import GlorotNormal
from tensorflow.keras.regularizers import l2

In [ ]:
def load_images_and_create_map(directory, plant_type_mapping):
    img_label_map = {}

    label_encoder = LabelEncoder()
    label_encoder.fit(list(plant_type_mapping.keys()))


    for plant_type, disease_folders in plant_type_mapping.items():

        for disease_folder in disease_folders:
            disease_folder_path = os.path.join(directory, disease_folder)

            for img_name in os.listdir(disease_folder_path):
                img_path = os.path.join(disease_folder_path, img_name)

                if os.path.isfile(img_path):

                    label = label_encoder.transform([plant_type])[0]

                    img_label_map[img_path] = label

    return img_label_map

In [ ]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(256, 256))
    img_array = img_to_array(img)
    img_array = img_array / 255.0
    return img_array

In [ ]:
train_dir = '/kaggle/input/full-data-final/ProjectData/Train/images/'
val_dir = '/kaggle/input/full-data-final/ProjectData/Val/images/'

In [ ]:
plant_type_mapping = {
    "Apple": ["Apple___Apple_scab", "Apple___Black_rot", "Apple___Cedar_apple_rust", "Apple___healthy"],
    "Cherry_(including_sour)": ["Cherry_(including_sour)___healthy", "Cherry_(including_sour)___Powdery_mildew"],
    "Corn_(maize)": ["Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot", "Corn_(maize)___Common_rust_", "Corn_(maize)___healthy", "Corn_(maize)___Northern_Leaf_Blight"],
    "Grape": ["Grape___Black_rot", "Grape___Esca_(Black_Measles)", "Grape___healthy", "Grape___Leaf_blight_(Isariopsis_Leaf_Spot)"],
    "Peach": ["Peach___Bacterial_spot", "Peach___healthy"],
    "Pepper,_bell": ["Pepper,_bell___Bacterial_spot", "Pepper,_bell___healthy"],
    "Potato": ["Potato___Early_blight", "Potato___healthy", "Potato___Late_blight"],
    "Strawberry": ["Strawberry___healthy", "Strawberry___Leaf_scorch"],
    "Tomato": ["Tomato___Bacterial_spot", "Tomato___Early_blight", "Tomato___healthy", "Tomato___Late_blight",
               "Tomato___Leaf_Mold", "Tomato___Septoria_leaf_spot", "Tomato___Spider_mites Two-spotted_spider_mite",
               "Tomato___Target_Spot", "Tomato___Tomato_mosaic_virus", "Tomato___Tomato_Yellow_Leaf_Curl_Virus"]
}

In [ ]:
train_img_label_map = load_images_and_create_map(train_dir, plant_type_mapping)
print("Training dataset mapping length:", len(train_img_label_map))

val_img_label_map = load_images_and_create_map(val_dir, plant_type_mapping)
print("Validation dataset mapping length:", len(val_img_label_map))

### **Mask Data**

In [ ]:
train_dir_mask = "/kaggle/input/full-data-final/ProjectData/Train/mask"
val_dir_mask = "/kaggle/input/full-data-final/ProjectData/Val/mask"

In [ ]:
train_img_label_map_mask = load_images_and_create_map(train_dir_mask, plant_type_mapping)
print("Training dataset mapping length:", len(train_img_label_map_mask))

val_img_label_map_mask = load_images_and_create_map(val_dir_mask, plant_type_mapping)
print("Validation dataset mapping length:", len(val_img_label_map_mask))

#### **Model**

In [ ]:
def enable_gpu():
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
        except RuntimeError as e:
            print(e)

enable_gpu()

In [ ]:
def preprocess_mask(mask_path):
    mask = load_img(mask_path, target_size=(256, 256), color_mode='grayscale')
    mask = img_to_array(mask).astype(np.float32)
    mask = mask / 255.0
    mask = np.where(mask > 0.5, 1, 0)
    return mask

In [ ]:
class SegmentationGenerator(Sequence):
    def __init__(self, image_paths, mask_paths, batch_size=16, img_size=(256, 256), num_classes=1):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.batch_size = batch_size
        self.img_size = img_size
        self.num_classes = num_classes

    def __len__(self):
        return len(self.image_paths) // self.batch_size

    def __getitem__(self, idx):
        batch_image_paths = self.image_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_mask_paths = self.mask_paths[idx * self.batch_size:(idx + 1) * self.batch_size]

        images = [preprocess_image(p) for p in batch_image_paths]
        masks = [preprocess_mask(p) for p in batch_mask_paths]

        return np.array(images), np.array(masks)
        return np.array(images), np.array(masks)

In [ ]:
def build_segmentation_model(input_shape=(256, 256, 3), num_classes=1):

    inputs = Input(shape=input_shape)
    backbone = ResNet50(weights='imagenet', include_top=False, input_tensor=inputs)

    x = backbone.get_layer('conv4_block6_out').output

    aspp = Conv2D(256, 1, padding="same", activation="relu")(x)
    aspp = UpSampling2D(size=(4, 4), interpolation='bilinear')(aspp)

    low_level_features = backbone.get_layer('conv2_block3_out').output
    low_level_features = Conv2D(48, 1, padding="same", activation="relu")(low_level_features)

    combined = Concatenate()([aspp, low_level_features])
    x = Conv2D(256, 3, padding="same", activation="relu")(combined)
    x = UpSampling2D(size=(4, 4), interpolation='bilinear')(x)

    outputs = Conv2D(1, 1, activation="sigmoid", padding="same")(x)

    return Model(inputs, outputs)

In [ ]:
model = build_segmentation_model()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy"])

model.summary()

In [ ]:
train_image_paths = sorted(train_img_label_map.keys())
train_mask_paths = sorted(train_img_label_map_mask.keys())

val_image_paths = sorted(val_img_label_map.keys())
val_mask_paths = sorted(val_img_label_map_mask.keys())

train_gen = SegmentationGenerator(train_image_paths, train_mask_paths, batch_size=16, num_classes=1)
val_gen = SegmentationGenerator(val_image_paths, val_mask_paths, batch_size=16, num_classes=1)

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    patience=8,
    monitor='val_accuracy',
    restore_best_weights=True)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20,
    callbacks=[early_stopping])

#model.save("segmentation_model.h5")

In [ ]:
val_loss, val_acc = model.evaluate(val_gen)
train_loss, train_acc = model.evaluate(train_gen)
print(f"\nFinal Validation Loss: {val_loss:.4f}, Final Validation Accuracy: {val_acc:.4f}")
print(f"\nFinal Training Loss: {train_loss:.4f}, Final Training Accuracy: {train_acc:.4f}")

In [ ]:
def predict_and_plot(image_path, mask_path, model, threshold=0.5):

    image = preprocess_image(image_path)
    true_mask = preprocess_mask(mask_path)

    image_input = np.expand_dims(image, axis=0)

    prediction = model.predict(image_input)
    predicted_mask = (prediction[0] > threshold).astype(np.uint8).squeeze()

    plt.figure(figsize=(18, 6))
    plt.subplot(1, 3, 1)
    plt.imshow(image)
    plt.title("Original Image")
    plt.axis("off")

    plt.subplot(1, 3, 2)
    plt.imshow(true_mask.squeeze(), cmap="gray")
    plt.title("True Mask")
    plt.axis("off")

    plt.subplot(1, 3, 3)
    plt.imshow(predicted_mask, cmap="gray")
    plt.title("Predicted Mask")
    plt.axis("off")

    plt.tight_layout()
    plt.show()


print("Model loaded successfully.")

image_path = "/kaggle/input/full-data-final/ProjectData/Train/images/Apple___Apple_scab/0208f4eb-45a4-4399-904e-989ac2c6257c___FREC_Scab 3037.JPG"
mask_path = "/kaggle/input/full-data-final/ProjectData/Train/mask/Apple___Apple_scab/0208f4eb-45a4-4399-904e-989ac2c6257c___FREC_Scab 3037_final_masked.jpg"
predict_and_plot(image_path, mask_path, model)

In [ ]:
image_path = "/kaggle/input/full-data-final/ProjectData/Val/images/Cherry_(including_sour)___healthy/4f870c34-1889-429c-adb2-0f70faf0ca30___JR_HL 9745.JPG"
mask_path = "/kaggle/input/full-data-final/ProjectData/Val/mask/Cherry_(including_sour)___healthy/4f870c34-1889-429c-adb2-0f70faf0ca30___JR_HL 9745_final_masked.jpg"

predict_and_plot(image_path, mask_path, model)

In [ ]:
image_path = "/kaggle/input/full-data-final/ProjectData/Val/images/Strawberry___healthy/096ce0d4-c5ef-4b66-b20f-f9ff62d09f48___RS_HL 1879.JPG"
mask_path = "/kaggle/input/full-data-final/ProjectData/Val/mask/Strawberry___healthy/096ce0d4-c5ef-4b66-b20f-f9ff62d09f48___RS_HL 1879_final_masked.jpg"

predict_and_plot(image_path, mask_path, model)

## **Test Script**

In [ ]:
import os
from sklearn.preprocessing import LabelEncoder

def load_images_and_create_map_test(directory, plant_type_mapping):

    img_label_map = {}

    label_encoder = LabelEncoder()
    label_encoder.fit(list(plant_type_mapping.keys()))

    for plant_type, disease_folders in plant_type_mapping.items():
        for disease_folder in disease_folders:
            disease_folder_path = os.path.join(directory, disease_folder)

            try:
                if not os.path.isdir(disease_folder_path):
                    continue  # Skip to the next disease folder

                for img_name in os.listdir(disease_folder_path):
                    img_path = os.path.join(disease_folder_path, img_name)

                    if os.path.isfile(img_path):
                        try:
                            label = label_encoder.transform([plant_type])[0]
                            img_label_map[img_path] = label
                        except Exception as e:
                            print(f"Error encoding label for plant type '{plant_type}': {e}")
                            continue
            except Exception as e:
                print(f"Error processing folder '{disease_folder_path}': {e}")
                continue

    return img_label_map


In [ ]:
test_dir = '/kaggle/input/segment/test seg/images'
mask_test_dir = "/kaggle/input/segment/test seg/binary_seg"

test_img_label_map = load_images_and_create_map_test(test_dir, plant_type_mapping)
print("Test dataset mapping length:", len(test_img_label_map))

test_img_label_map_mask = load_images_and_create_map_test(mask_test_dir, plant_type_mapping)
print("Test dataset mapping mask length:", len(test_img_label_map_mask))


In [ ]:
test_image_paths = sorted(test_img_label_map.keys())
test_mask_paths = sorted(test_img_label_map_mask.keys())


test_gen = SegmentationGenerator(test_image_paths, test_mask_paths, batch_size=16, num_classes=1)


In [ ]:
model_path = "/kaggle/input/segmentation_model/other/default/1/segmentation_model.h5"
#model = load_model(model_path, compile=False)
print("Model loaded successfully.")

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix

y_pred = model.predict(test_gen, verbose=1)
y_pred_binary = (y_pred > 0.5).astype(np.float32)

y_true_list = []
y_pred_list = []
current_index = 0

for i in range(len(test_gen)):

    _, masks = test_gen[i]
    y_true_list.append(masks)

    batch_size = masks.shape[0]
    #print("batch size : " , batch_size)

    y_pred_list.append(y_pred_binary[current_index:current_index + batch_size])

    current_index += batch_size

y_true = np.concatenate(y_true_list, axis=0).reshape(-1)
y_pred_flat = np.concatenate(y_pred_list, axis=0).reshape(-1)

print(f"Length of y_true: {len(y_true)}")
print(f"Length of y_pred_flat: {len(y_pred_flat)}")

accuracy = accuracy_score(y_true, y_pred_flat)
print(f"Test Pixel-wise Accuracy: {accuracy:.4f}")

cm = confusion_matrix(y_true, y_pred_flat)
print("Confusion Matrix:")
print(cm)
